In [ ]:
**Instrucciones**


Dado:

* Habitación de MxN espacios.
* Número de agentes.
* Porcentaje de celdas inicialmente sucias.
* Tiempo máximo de ejecución.


Realiza la siguiente simulación:

* Inicializa las celdas sucias (ubicaciones aleatorias).
* Todos los agentes empiezan en la celda [1,1].
* En cada paso de tiempo:
-- Si la celda está sucia, entonces aspira.
-- Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
* Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución:

* Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
* Porcentaje de celdas limpias después del termino de la simulación.
* Número de movimientos realizados por todos los agentes.


SyntaxError: ignored

In [ ]:
!pip install mesa

from mesa import Agent, Model

from mesa.space import MultiGrid

from mesa.time import SimultaneousActivation

from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import time
import datetime
import random

In [16]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
      cell_content, x, y = cell
      for obj in cell_content:
        if isinstance(obj, RobotLimpieza):
          grid[x][y] = 2
        elif isinstance(obj, Celda):
          grid[x][y] = obj.estado
    return grid


class RobotLimpieza(Agent):
  '''
  Aquí definimos al agente Robot
  '''
  def __init__(self, unique_id, model): # Init = constructor de la clase
    super().__init__(unique_id, model)
    self.sig_pos = None
    self.contPasos = 0

  def step(self):
    # print('Estoy en el step ')
    vecinos = self.model.grid.get_neighbors(
        self.pos,
        moore = True,
        include_center = True)
    
    # Máquina de estados
    for vecino in vecinos:
      if isinstance(vecino, Celda) and self.pos == vecino.pos:
        if vecino.estado == 1:
          # Limpiar
          vecino.sig_estado = 0
          self.sig_pos = self.pos
        else: # vecino.estado == 0 (limpio)
          # Escoger posicion a moverse
          vecinos_2 = self.model.grid.get_neighborhood(
            self.pos,
            moore = True, # Moore incluye vecinos en diagonal
            include_center = False)
          vecino.sig_estado = 0
          self.sig_pos = self.random.choice(vecinos_2)
        break

    #
  def advance(self):
    vecinos = self.model.grid.get_neighbors(
      self.pos,
      moore = True,
      include_center = True)
    for vecino in vecinos:
      if isinstance(vecino, Celda) and self.pos == vecino.pos:
        vecino.estado = vecino.sig_estado
    self.model.grid.move_agent(self, self.sig_pos)

class Celda(Agent):
  # 1 = Sucio
  # 0 = Limpio
  '''
  Aquí definimos al agente celda
  '''
  def __init__(self, unique_id, model, estado):
    super().__init__(unique_id, model)
    self.pos = unique_id
    self.estado = estado
    self.sig_estado = None


class Habitacion(Model):

  def __init__(self, M, N, num_agentes, porc_celdas_sucias):
    self.num_agentes = num_agentes
    self.porc_celdas_sucias = porc_celdas_sucias
    self.porc_celdas_limpias = 1 - porc_celdas_sucias
    self.grid = MultiGrid(M, N, False) # Torus permite que los agentes puedan brincar de un extremo del grid al otro (Es un boleano)
    self.schedule = SimultaneousActivation(self)

    # Colocación de celdas limpias/sucias
    num_celdas_sucias = int(M*N*porc_celdas_sucias)
    for(content, x, y) in self.grid.coord_iter():
      num = random.randint(0, 1)
      if num == 1 and num_celdas_sucias > 0:
        a = Celda((x, y), self, 1)
        num_celdas_sucias -= 1
      else:
        a = Celda((x, y), self, 0)

      # a = Celda((x, y), self, num_celdas_sucias)
      self.grid.place_agent(a, (x, y))
      self.schedule.add(a)

    # Colocación de agentes robots
    for id in range(num_agentes):
      r = RobotLimpieza(id, self)
      self.grid.place_agent(r, (1, 1))
      self.schedule.add(r)

    self.datacollector = DataCollector(
        model_reporters = {"Grid": get_grid})


  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()

  def todoLimpio(self):
    for(content, x, y) in self.grid.coord_iter():
      for obj in content:
        if isinstance(obj, Celda) and obj.estado == 1:
          return False
    return True

In [17]:
M = 15
N = 10
num_agentes = 5
porc_celdas_sucias = 0.6
tiempoMax = 0.5

model = Habitacion(M, N, num_agentes, porc_celdas_sucias)
start_time = time.time()
while((time.time() - start_time) < tiempoMax and not model.todoLimpio()):
  model.step()

print('Tiempo de ejecución: ', str(datetime.timedelta(seconds = (time.time() - start_time))))


Tiempo de ejecución:  0:00:00.096359


In [18]:
all_grid = model.datacollector.get_model_vars_dataframe()
#print(all_grid.to_string())

In [19]:
%%capture

fig, axs = plt.subplots(figsize = (7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap = plt.cm.binary)

def animate(i):
  patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames = len(all_grid))

In [20]:
anim